In [ ]:
%run -i individual.py
%run -i train.py

In [ ]:
from keras.utils import plot_model
from operator import attrgetter
from copy import deepcopy

def evalOneMax(individual, g, ngen, nth, is_final=False):
  model, score = _main_(individual, is_final)
  plot_model(model, to_file=getModelName(g, nth, score),
             show_shapes=True, show_layer_names=False)
  return score,

def getModelName(gen, nth, score):
  return f'models/{gen + 1:0>2}-{nth:0>2}_{score}.png'

def mutCNNStructure(individual):
  func_list = [
    add_convolution,
    remove_convolution,
    alter_channel_number,
    alter_filter_size,
    alter_stride,
    add_dropout,
    remove_dropout,
    add_pooling,
    remove_pooling,
    add_skip,
    remove_skip,
    add_concatenate,
    remove_concatenate,
    add_fully_connected,
    remove_fully_connected
  ]
  func_weights = [
    2, 1, 2, 2, 2,
    1, 1, 1, 1, 2,
    1, 2, 1, 1, 1
  ]

  func = random.choices(func_list, k=1, weights=func_weights)[0]
  while True:
    try:
      individual.last_applied_operation = func.__name__
      func(individual)
    except RuntimeError:
      import traceback
      traceback.print_exc()
      func = random.choices(func_list, k=1, weights=func_weights)[0]
    else:
      break
  return individual,

def selAggressive(individuals, fit_attr="fitness"):
  top_1 = max(individuals, key=attrgetter(fit_attr))
  chosen = [deepcopy(top_1) for _ in individuals]
  return chosen

def adaptiveNumEpochs(g, ngen):
  return ((ngen - g) * min_epochs + g * max_epochs) // ngen

In [ ]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.


#    example which maximizes the sum of a list of integers
#    each of which can be 0 or 1

import random
import csv
from datetime import datetime

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", Individual, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, creator.Individual)

#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evalOneMax)

# register a mutation operator
toolbox.register("mutate", mutCNNStructure)

# register the aggressive selection
toolbox.register("select", selAggressive)

#----------

def getStats(pop, fit_attr="fitness"):
    fits = [ind.fitness.values[0] for ind in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    timestamp = str(datetime.now())

    top_1 = max(pop, key=attrgetter(fit_attr))
    best_operation = top_1.last_applied_operation

    return [best_operation, min(fits), max(fits), mean, std, timestamp]

def main():
    random.seed(64)

    # create an initial population of 10 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=3)
    [add_convolution(ind) for ind in pop]
    [add_convolution(ind) for ind in pop]
    [add_convolution(ind) for ind in pop]
    NGEN = 5

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = [toolbox.evaluate(ind, 0, NGEN, 99) for ind in pop]
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    f = open('result.csv', 'w')
    writer = csv.writer(f)
    headers = ['gen', 'operations', 'min', 'max', 'mean', 'std', 'timestamp']
    writer.writerow(headers)
    writer.writerow([0] + getStats(pop))

    print("  Evaluated %i individuals" % len(pop))

    # Begin the evolution
    for g in range(NGEN):
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop)
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        for mutant in offspring:
            if mutant is offspring[0]:
                mutant.last_applied_operation = 'do_nothing'
                continue
            else:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = [toolbox.evaluate(ind, g, NGEN, i) for i, ind in enumerate(invalid_ind)]
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        writer.writerow([g + 1] + getStats(pop))

    print("-- End of (successful) evolution --")
    f.close()

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is")
    toolbox.evaluate(best_ind, NGEN, NGEN, 99, True)

In [ ]:
main()